In [25]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
import gensim 
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [26]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

In [27]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path(".")

In [28]:
my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

In [29]:
data = list()
for i in documents: 
    temp = list()
    # tokenize the sentence into words 
    for j in word_tokenize(i[0]): 
        temp.append(j.lower()) 
    data.append(temp) 

In [30]:
cbow_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5,sg = 3)

In [31]:
def preprocess(text):
    tokens = []
    for word in word_tokenize(text, language='english'):
        if len(word) >= 2 and word not in stop_words:
            tokens.append(word)
    return tokens

In [32]:
len(documents)

2378

In [33]:
preprocess(documents[0][0])

['ontario',
 'automobile',
 'policy',
 'oap',
 'owner',
 'policy',
 'approved',
 'superintendent',
 'financial',
 'services',
 'use',
 'standard',
 'owner',
 "'s",
 'policy',
 'june',
 '2016',
 'policy',
 'automobile',
 'insurance',
 'policy.it',
 'written',
 'easy',
 'understand',
 'language.please',
 'read',
 'carefully',
 'know',
 'rights',
 'obligations',
 'rights',
 'obligations',
 'insurance',
 'company']

In [34]:
# Define a function to generate text embeddings
def text_embedding(text, model):
    # Preprocess the text (tokenize, remove stop words, stem)
    preprocessed_text = preprocess(text)
    # Get the word embeddings for each word in the document
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    # Take the average of the word embeddings to generate a document embedding
    if len(word_embeddings) > 0:
        document_embedding = np.mean(word_embeddings, axis=0)
    else:
        document_embedding = np.zeros(model.vector_size)
    return document_embedding

In [35]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [36]:
document_vecs = [text_embedding(document[0], cbow_model) for document in documents]

In [37]:
pd.DataFrame(document_vecs)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.206760,0.075865,-0.026819,-0.040559,-0.048716,-0.482175,0.271679,0.068147,0.147383,0.038855,...,0.363528,-0.140106,0.189267,-0.153646,0.079023,-0.027334,0.174552,-0.267479,0.205350,0.157894
1,0.191204,0.260320,-0.043990,-0.093248,-0.026147,-0.516109,0.312099,-0.124343,0.126783,0.051251,...,0.343220,-0.064934,0.125365,-0.300347,-0.066709,-0.057929,0.379724,-0.204030,0.242899,-0.033553
2,0.153436,0.153806,-0.035261,-0.199692,-0.064613,-0.435204,0.134580,-0.162544,0.235665,0.012536,...,0.431576,0.057680,0.136943,-0.269419,0.029100,0.033866,0.379266,-0.174580,0.280272,0.118078
3,0.202942,0.179945,0.011387,-0.184616,-0.136863,-0.444898,0.169176,-0.181898,0.203954,0.051245,...,0.524151,-0.117787,0.100583,-0.264092,0.076176,0.102506,0.343104,-0.150088,0.155642,0.029397
4,0.159498,0.224403,0.005636,-0.202153,-0.120232,-0.435317,0.240811,-0.202509,0.117135,0.061005,...,0.327758,-0.071945,0.123959,-0.314531,0.049050,0.086094,0.403940,-0.143392,0.244617,0.050653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,0.141287,0.044903,-0.026288,-0.009997,0.049393,-0.319919,0.339456,0.117913,0.074017,0.046852,...,0.379170,-0.069886,0.215072,-0.025144,0.160938,0.023463,0.164428,-0.184987,0.191127,0.014332
2374,0.093684,0.020863,0.036180,0.005621,0.027795,-0.423193,0.303999,0.072533,0.166863,-0.008424,...,0.379880,-0.109237,0.200014,-0.147107,0.159144,0.128955,0.111707,-0.207321,0.114990,0.048363
2375,0.107748,0.025973,0.008374,-0.023413,0.070776,-0.431803,0.310500,0.034714,0.134016,-0.021332,...,0.360565,-0.060158,0.259109,-0.138417,0.217536,0.079199,0.139054,-0.250929,0.161964,0.040374
2376,-0.004858,0.017449,0.093048,0.068272,0.051856,-0.320239,0.262563,0.112322,0.042014,0.022781,...,0.167658,-0.123013,0.076337,-0.055742,0.119773,0.181751,0.086742,-0.205860,0.014178,-0.006759


In [38]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'wb')
pickle.dump(document_vecs, dbfile) 
dbfile.close()

In [39]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'rb')     
document_vecs = pickle.load(dbfile)
dbfile.close()

In [47]:
# Example usage: compare a query with a set of documents
query = "If you are a corporation, unincorporated association, partnership, sole"

query_vec = text_embedding(query,cbow_model)
# Calculate similarity scores between the query and all documents
similarity_scores = [cosine_similarity(query_vec, document_vec) for document_vec in document_vecs]
# Rank documents based on similarity scores
ranked_documents = [document for _, document in sorted(zip(similarity_scores, documents), reverse=True)]
# Print the ranked documents
ranked_documents[0:5]

[("arising out of any insured 's : entrustment to any other person ; vicarious parental liability , whether or not imposed by statute , for the actions of a child or minor relating to the maintenance , use , loading or unloading ; or supervision of any person in the maintenance , use , loading or unloading ; of motorized vehicles , including trailers of any type , watercraft or aircraft.arising out of the ownership of , custody of , or the care for a pure or mixed breed american pit bull terrier ( also known as an american staffordshire terrier , pit bull terrier or pit bull ) , rottweiler or canary ( also known as a presa canario or a perro de presa canario ) dog.loss assessment we will pay up to $ 1,000 for your share of loss assessment charged during the policy period against you by a corporation or association of property owners , when the assessment is made as a result of : bodily injury or property damage not excluded under section ii of this policy ; or liability for an act of a